In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
# pip install tensorflow ignore if already installed

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [4]:
dataset = pd.read_csv('Churn_Modelling.csv', index_col = 'RowNumber')

In [5]:
dataset.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
# Features and target
X = dataset.iloc[:, 2:12]
X  # drop CustomerId and Surname

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
RowNumber,,,,,,,,,,
1,619,France,Female,42,2,0.00,1,1,1,101348.88
2,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
3,502,France,Female,42,8,159660.80,3,1,0,113931.57
4,699,France,Female,39,1,0.00,2,0,0,93826.63
5,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9996,771,France,Male,39,5,0.00,2,1,0,96270.64
9997,516,France,Male,35,10,57369.61,1,1,1,101699.77
9998,709,France,Female,36,7,0.00,1,0,1,42085.58


In [7]:
# Features and target
Y = dataset.iloc[:, 12].values  # Churn column
Y

array([1, 0, 1, ..., 1, 1, 0])

We are treating countries with ordinal values(0 < 1 < 2) but they are incomparable.
To solve this we can use one hot encoding.
We will perform some standardization

In [8]:
# Pipeline: One-hot encode categorical and scale
pipeline = Pipeline([
    ('preprocess', ColumnTransformer(
        transformers=[
            ('gender', OneHotEncoder(drop='first'), ['Gender']),
            ('geo', OneHotEncoder(drop='first'), ['Geography'])
        ],
        remainder='passthrough'
    )),
    ('scaler', StandardScaler())
])

In [9]:
#Standardize the features
X = pipeline.fit_transform(X)

In [10]:
#Spilt the data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [11]:
# Build ANN
classifier = Sequential()
classifier.add(Dense(6, activation='relu', input_shape=(X_train.shape[1],)))
classifier.add(Dropout(0.1))
classifier.add(Dense(6, activation='relu'))
classifier.add(Dropout(0.1))
classifier.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Compile ANN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# Train ANN
history = classifier.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.1, verbose=2)

Epoch 1/100
225/225 - 2s - 9ms/step - accuracy: 0.7961 - loss: 0.5369 - val_accuracy: 0.7950 - val_loss: 0.4721
Epoch 2/100
225/225 - 0s - 2ms/step - accuracy: 0.7961 - loss: 0.4891 - val_accuracy: 0.7950 - val_loss: 0.4478
Epoch 3/100
225/225 - 0s - 2ms/step - accuracy: 0.7961 - loss: 0.4713 - val_accuracy: 0.7950 - val_loss: 0.4334
Epoch 4/100
225/225 - 0s - 2ms/step - accuracy: 0.7961 - loss: 0.4552 - val_accuracy: 0.7950 - val_loss: 0.4234
Epoch 5/100
225/225 - 0s - 2ms/step - accuracy: 0.7961 - loss: 0.4498 - val_accuracy: 0.7950 - val_loss: 0.4162
Epoch 6/100
225/225 - 0s - 2ms/step - accuracy: 0.7961 - loss: 0.4408 - val_accuracy: 0.7950 - val_loss: 0.4099
Epoch 7/100
225/225 - 0s - 2ms/step - accuracy: 0.7961 - loss: 0.4334 - val_accuracy: 0.7950 - val_loss: 0.4056
Epoch 8/100
225/225 - 1s - 2ms/step - accuracy: 0.7961 - loss: 0.4308 - val_accuracy: 0.7950 - val_loss: 0.4025
Epoch 9/100
225/225 - 0s - 2ms/step - accuracy: 0.7961 - loss: 0.4301 - val_accuracy: 0.7950 - val_loss:

In [15]:
# Predict
y_pred = (classifier.predict(X_test) > 0.5).astype(int)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


In [16]:
y_pred = classifier.predict(X_test)
print(y_pred[:5])

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
[[0.29576513]
 [0.27789894]
 [0.17470004]
 [0.0738398 ]
 [0.13504124]]


In [17]:
#Let us use confusion matrix with cutoff value as 0.5
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [18]:
#Making the Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1527   68]
 [ 215  190]]


In [19]:
#Accuracy of our NN
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

85.85 % of data was classified correctly
